In [2]:
import pymssql
import configparser
import numpy as np
import pandas as pd

In [3]:
config = configparser.ConfigParser()
config.read('db_config.ini')

db_host = config['database']['host']
db_user = config['database']['user']
db_password = config['database']['password']
db_name = config['database']['dbname']

sql_conn = pymssql.connect(db_host, db_user, db_password, db_name)
cursor = sql_conn.cursor(as_dict=True)

In [4]:
def get_data(sql_conn, query):
    cursor = sql_conn.cursor(as_dict=True)
    cursor.execute(query)
    results = cursor.fetchall()
    columns = [column[0] for column in cursor.description]
    df = pd.DataFrame(results, columns=columns)
    return df

In [5]:
def run_query(sql_conn, query):
    cursor = sql_conn.cursor(as_dict=True)
    cursor.execute(query)
    return None

In [6]:
query = """
SELECT [person_id]
      ,MIN([condition_start_date]) AS [first_visit_date]
      ,MAX([condition_start_date]) AS [last_visit_date]
      ,COUNT([visit_occurrence_id]) AS [n_visits]
FROM [condition_occurrence]
WHERE [condition_concept_id] IN (35207170)
GROUP BY [person_id]
HAVING COUNT([visit_occurrence_id]) >= 1
"""
all_subjects = get_data(sql_conn, query)

In [ ]:
all_subjects.to_feather("all_subjects.feather")

In [7]:
subject_ids = all_subjects.person_id.tolist()
subject_ids = '), ('.join([str(subject_ids) for subject_ids in subject_ids])
subject_ids = '(' + subject_ids + ')'
subject_ids = subject_ids.split(', ')

In [10]:
query = f"""
CREATE TABLE #temp_person_ids (person_id INT);
"""

for subject_id in subject_ids:
    query += f"""
INSERT INTO #temp_person_ids (person_id)
VALUES {subject_id};
"""
run_query(sql_conn, query)

In [11]:
query = f"""
SELECT *
FROM [OMOP_DEID].[dbo].[condition_occurrence]
WHERE [person_id] IN (SELECT person_id FROM #temp_person_ids);
"""
conditions = get_data(sql_conn, query)

In [ ]:
conditions.to_feather("conditions.feather")

In [ ]:
query = f"""
SELECT *
FROM [OMOP_DEID].[dbo].[drug_exposure]
WHERE [person_id] IN (SELECT person_id FROM #temp_person_ids);
"""
drugs = get_data(sql_conn, query)

In [ ]:
drugs.to_feather("conditions.feather")

In [12]:
query = f"""
SELECT *
FROM [OMOP_DEID].[dbo].[person]
WHERE [person_id] IN (SELECT person_id FROM #temp_person_ids);
"""
subject_details = get_data(sql_conn, query)

In [15]:
subject_details.to_feather("subject_details.feather")